In [9]:
import pandas as pd
import json
from pymongo import MongoClient
import mysql.connector
import time
from sqlalchemy import create_engine,text
from sqlalchemy.exc import PendingRollbackError
import kagglehub

In [10]:
# load configuration file with password for mongoDB and mySQL
with open('config.json', 'r') as f:
    config = json.load(f)

# Extract username and password for MongoDB and MySQL
mongo_username = config["mongo"]["username"]
mongo_password = config["mongo"]["password"]
mongo_host = config["mongo"]["host"]
mongo_port = config["mongo"]["port"]
mysql_username = config["mysql"]["username"]
mysql_password = config["mysql"]["password"]
mysql_host = config["mysql"]["host"]
mysql_port = config["mysql"]["port"]

In [ ]:
# Download latest dataset version for football matches
# path = kagglehub.dataset_download("martj42/international-football-results-from-1872-to-2017")

# Download latest dataset version for books recommendation
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

In [13]:
# Load dataset into pandas dataframe
df_goalscorers = pd.read_csv(f'{path}/goalscorers.csv')
df_results = pd.read_csv(f'{path}/goalscorers.csv')
df_shootouts = pd.read_csv(f'{path}/goalscorers.csv')

In [14]:
# Connect to MongoDB locally
client = MongoClient(f'mongodb://{mongo_host}:{mongo_port}',
                             username = mongo_username,
                             password = mongo_password)

client.drop_database("project")

db = client["project"]

db["goalscorers"].drop()
db["results"].drop()
db["shootouts"].drop()

goalscorers = db["goalscorers"]
results = db["results"]
shootouts = db["shootouts"]

goalscorers.insert_many(df_goalscorers.to_dict(orient="records"), ordered=False)
results.insert_many(df_results.to_dict(orient="records"), ordered=False)
shootouts.insert_many(df_shootouts.to_dict(orient="records"), ordered=False)

InsertManyResult([ObjectId('6731de91890c9fb51d039bf0'), ObjectId('6731de91890c9fb51d039bf1'), ObjectId('6731de91890c9fb51d039bf2'), ObjectId('6731de91890c9fb51d039bf3'), ObjectId('6731de91890c9fb51d039bf4'), ObjectId('6731de91890c9fb51d039bf5'), ObjectId('6731de91890c9fb51d039bf6'), ObjectId('6731de91890c9fb51d039bf7'), ObjectId('6731de91890c9fb51d039bf8'), ObjectId('6731de91890c9fb51d039bf9'), ObjectId('6731de91890c9fb51d039bfa'), ObjectId('6731de91890c9fb51d039bfb'), ObjectId('6731de91890c9fb51d039bfc'), ObjectId('6731de91890c9fb51d039bfd'), ObjectId('6731de91890c9fb51d039bfe'), ObjectId('6731de91890c9fb51d039bff'), ObjectId('6731de91890c9fb51d039c00'), ObjectId('6731de91890c9fb51d039c01'), ObjectId('6731de91890c9fb51d039c02'), ObjectId('6731de91890c9fb51d039c03'), ObjectId('6731de91890c9fb51d039c04'), ObjectId('6731de91890c9fb51d039c05'), ObjectId('6731de91890c9fb51d039c06'), ObjectId('6731de91890c9fb51d039c07'), ObjectId('6731de91890c9fb51d039c08'), ObjectId('6731de91890c9fb51d039c

In [15]:
# Connect to mySQL locally
db_url = f'mysql+mysqlconnector://{mysql_username}:{mysql_password}@{mysql_host}:{mysql_port}'
engine = create_engine(db_url)

database_name = 'project'
table_names = ['goalscorers', 'results', 'shootouts']

with engine.connect() as connection:
    connection.execute(text(f"DROP DATABASE IF EXISTS {database_name}"))
    connection.execute(text(f"CREATE DATABASE {database_name}"))

engine.dispose()

db_url = f'mysql+mysqlconnector://{mysql_username}:{mysql_password}@{mysql_host}:{mysql_port}/{database_name}'
engine = create_engine(db_url)

try:
    df_goalscorers.to_sql(table_names[0], con=engine, if_exists='replace', index=False, method='multi', chunksize=1000)
    df_results.to_sql(table_names[1], con=engine, if_exists='replace', index=False, method='multi', chunksize=1000)
    df_shootouts.to_sql(table_names[2], con=engine, if_exists='replace', index=False, method='multi', chunksize=1000)
    print("Data inserted successfully!")

finally:
    engine.dispose()

Data inserted successfully!


In [16]:
with engine.connect() as connection:
    #result = connection.execute(text("SELECT * FROM goalscorers WHERE penalty = TRUE"))
    #for row in result:
    #    print(row)

    #result = connection.execute(text("SELECT * FROM results WHERE home_team = 'Scotland'"))
    #for row in result:
    #    print(row)

    result = connection.execute(text("SELECT * FROM shootouts WHERE home_team = first_shooter"))
    for row in result:
        print(row)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'first_shooter' in 'where clause'
[SQL: SELECT * FROM shootouts WHERE home_team = first_shooter]
(Background on this error at: https://sqlalche.me/e/20/f405)